In [1]:
from model import *
from data import *

Using TensorFlow backend.


## Train your Unet with medical data - Pelvis
it is a binary classification task.

The input shape of image and mask are the same :(batch_size,rows,cols,channel = 1)

### Set hyperparameters

* epochs_no - more runs and improvs generalisation of the model
* batch_size - number of samples to run at each epoch step

In [2]:
# Hyper parametrs
batch_size = 8
epochs_no = 2000
steps_per_epoch = epochs_no / batch_size
epochs_no = 50

### Model Definition

In [3]:
input_size = (256,256,1) # actual data
#input_size = (256,256,1) # Augmented data
model = unet(input_size=input_size)
model_checkpoint = ModelCheckpoint('unet_pelvis.hdf5', monitor='loss',verbose=1, save_best_only=True)

D:\Research\ML\unet_ml\model.py:55: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("co..., inputs=Tensor("in...)`
  model = Model(input = inputs, output = conv10)


In [ ]:
# callbacks = [ModelCheckpoint(filepathAcc, monitor='val_acc', ...),ModelCheckpoint(filepathLoss, monitor='val_loss', ...)]

### Train with data generator

In [4]:
#if you don't want to do data augmentation, set data_gen_args as an empty dict.
#data_gen_args = dict()

data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')

In [5]:
# Actual data size ~train, label folders and uses augmented saved data in ~aug folder
myGene = trainGenerator(batch_size,'data/pelvis/train','image','label',data_gen_args,save_to_dir = None)

# Downsampled data and no augmentation
#myGene = trainGenerator(batch_size,'data/pelvis/train','image_downsampled','label_downsampled',data_gen_args,save_to_dir = None)

In [6]:
model.fit_generator(myGene,steps_per_epoch=steps_per_epoch,epochs=epochs_no,callbacks=[model_checkpoint])

Epoch 1/50
Found 98 images belonging to 1 classes.
Found 98 images belonging to 1 classes.
250/250 [==============================] - 202s 807ms/step - loss: 0.0438 - acc: 0.9861

Epoch 00001: loss improved from inf to 0.04453, saving model to unet_pelvis.hdf5
Epoch 2/50
250/250 [==============================] - 189s 756ms/step - loss: 0.0130 - acc: 0.9945

Epoch 00002: loss improved from 0.04453 to 0.01299, saving model to unet_pelvis.hdf5
Epoch 3/50
250/250 [==============================] - 190s 761ms/step - loss: 0.0102 - acc: 0.9957

Epoch 00003: loss improved from 0.01299 to 0.01025, saving model to unet_pelvis.hdf5
Epoch 4/50
250/250 [==============================] - 190s 758ms/step - loss: 0.0089 - acc: 0.9962

Epoch 00004: loss improved from 0.01025 to 0.00899, saving model to unet_pelvis.hdf5
Epoch 5/50
250/250 [==============================] - 190s 760ms/step - loss: 0.0082 - acc: 0.9965

Epoch 00005: loss improved from 0.00899 to 0.00817, saving model to unet_pelvis.hdf5

KeyboardInterrupt: 

### Train with npy file

### a. Train with Augmented data

In [ ]:
imgs_train,imgs_mask_train = geneTrainNpy("data/pelvis/train/aug/","data/pelvis/train/aug/")
#model.fit(imgs_train, imgs_mask_train, batch_size=batch_sizes, epochs=epochs_no, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])

### b. Train with Original data

In [ ]:
imgs_train,imgs_mask_train = geneTrainNpy("data/pelvis/train/image_downsampled/","data/pelvis/train/label_downsampled/", image_prefix="",mask_prefix="")
model.fit(imgs_train, imgs_mask_train, batch_size=batch_size, epochs=epochs_no, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])

### Test your model and save predicted results

Link to Stack overflow discussion - https://stackoverflow.com/questions/52946110/u-net-low-contrast-test-images-predict-output-is-grey-box

### Set the output data dimension here

input and target size must match!

In [7]:
## Resample to original sizes
input_size = (512,512,1)
target_size=(512,512)

### Test model on n datasets

In [ ]:
import os
test_path = 'data/pelvis/test_orig'
num_images = 0
for entry in os.listdir(test_path):
    if os.path.isfile(os.path.join(test_path, entry)):
        if entry.lower().endswith(('.png', '.jpg', '.jpeg')):
            num_images = num_images + 1

In [13]:
# Path toread datasets tp apply prediction to
testGene = testGenerator(test_path,num_image=num_images, target_size=target_size)

In [14]:
model = unet(input_size=input_size)
model.load_weights("unet_pelvis.hdf5")
results = model.predict_generator(testGene,num_images,verbose=1)

D:\Research\ML\unet_ml\model.py:55: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("co..., inputs=Tensor("in...)`
  model = Model(input = inputs, output = conv10)
D:\ProgramData\Anaconda3\envs\tensorflowGPU\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
D:\ProgramData\Anaconda3\envs\tensorflowGPU\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


52/52 [==============================] - 7s 142ms/step


### Save the predicted data

In [15]:
# Input is image and path to save the predicted datasets to
saveResult("data/pelvis/test_orig_label",results)

D:\ProgramData\Anaconda3\envs\tensorflowGPU\lib\site-packages\skimage\util\dtype.py:130: UserWarning: Possible precision loss when converting from float32 to uint8
  .format(dtypeobj_in, dtypeobj_out))
